In [1]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display

# ================= FUNCTIONS =================
def predict_contamination(ph, tds, hardness, nitrate):
    score = 0
    if ph < 6.5 or ph > 8.5: score += 30
    if tds > 500: score += 25
    if hardness > 200: score += 20
    if nitrate > 45: score += 25
    return score

# ================= USER INPUTS =================
ph = float(input("Enter pH Level (0-14): "))
tds = float(input("Enter TDS (mg/L): "))
hardness = float(input("Enter Hardness (mg/L): "))
nitrate = float(input("Enter Nitrate (mg/L): "))
location = input("Enter Location: ")

# ================= PREDICTION =================
score = predict_contamination(ph, tds, hardness, nitrate)
if score < 30:
    result = 'Safe'
elif score < 60:
    result = 'Moderate Risk'
else:
    result = 'High Risk'

print(f"\nContamination Risk: {result} ({score}%)")

# ================= GAUGE =================
fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=score,
    title={'text': "Radioactive Risk %"},
    gauge={
        'axis': {'range': [0, 100]},
        'bar': {'color': "red" if score >= 60 else "orange" if score >= 30 else "green"},
        'steps': [
            {'range': [0, 30], 'color': "green"},
            {'range': [30, 60], 'color': "yellow"},
            {'range': [60, 100], 'color': "red"}
        ],
    }
))
fig.show()

# ================= MINI DASHBOARD =================
safe_ranges = {
    "pH": (6.5, 8.5, ph),
    "TDS (mg/L)": (0, 500, tds),
    "Hardness (mg/L)": (0, 200, hardness),
    "Nitrate (mg/L)": (0, 45, nitrate)
}

for param, (low, high, value) in safe_ranges.items():
    status = "Safe" if low <= value <= high else "Unsafe"
    print(f"{param}: {value} (Safe Range: {low}-{high}) => {status}")

# ================= SAVE DATA =================
new_data = pd.DataFrame([[location, ph, tds, hardness, nitrate, score]],
                        columns=["Location", "pH", "TDS", "Hardness", "Nitrate", "RiskScore"])

try:
    old_data = pd.read_csv("water_data.csv")
    df = pd.concat([old_data, new_data], ignore_index=True)
except FileNotFoundError:
    df = new_data

df.to_csv("water_data.csv", index=False)
print("\nData saved to water_data.csv")


Enter pH Level (0-14): 6
Enter TDS (mg/L): 400
Enter Hardness (mg/L): 52
Enter Nitrate (mg/L): 24
Enter Location: avadi

Contamination Risk: Moderate Risk (30%)


pH: 6.0 (Safe Range: 6.5-8.5) => Unsafe
TDS (mg/L): 400.0 (Safe Range: 0-500) => Safe
Hardness (mg/L): 52.0 (Safe Range: 0-200) => Safe
Nitrate (mg/L): 24.0 (Safe Range: 0-45) => Safe

Data saved to water_data.csv
